In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
flagD = 0
th = 72
numStocCells = 30
STIMligs = [0.0,0.0,0.0,0.0,0.0,0.0,0.0] # No GM cond
Meki_ind = 768
Akti_ind = 770
inh_conc = 10000.0
nmxlsfile1 = 'MAi_v1_25cells_72hrs_' # No stim # THIS CONDITION WILL BE SIMULATED IN THIS NOTEBOOK
nmxlsfile2 = 'MAi_v2_25cells_72hrs_' # E+I
nmxlsfile3 = 'MAi_v3_25cells_72hrs_' # E+I+Meki
nmxlsfile4 = 'MAi_v4_25cells_72hrs_' # E+I+Akti
nmxlsfile5 = 'MAi_v5_25cells_72hrs_' # E+I+Meki+Akti

ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)
Vn = 1.75E-12
Vc = 5.25E-12

genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)
tout_all = np.arange(0,th*3600+1,30)

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints
model0 = model

currentDirectory = os.getcwd()

# Use pre-simulated stochastic cells without any ligands (24 hours, serum-starvation cond). 
# The end-point of each pre-simulated cell is the starting condition for one cell
namexlsS = 'NoGF_100cells_24of72hrs_S.txt' 
condsS0 = pd.read_csv(namexlsS,header=0,index_col=0,sep="\t")
spdataMat0 = condsS0.loc[:,1:] 
namexlsG = 'NoGF_100cells_24of72hrs_G.txt'
condsG0 = pd.read_csv(namexlsG,header=0,index_col=0,sep="\t")
genedataMat0 = condsG0.loc[:,1:]

startTime = datetime.now()
print(startTime)
for nn in range(numStocCells):
    # No Stim
    spdata = np.array(spdataMat0.loc[nn,:])
    spdata[155:162] = STIMligs
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata # 24hr time point     
    genedata = np.array(genedataMat0.loc[nn,:])
    xoutG_all = np.zeros(shape=(NSteps+1,len(genedata)))
    xoutG_all[0,:] = genedata
    model2 = model0
    solver2 = model2.getSolver() # Create solver instance
    solver2.setMaxSteps = 1e10
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model2.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model2.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model2, solver2)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        xoutG_all[qq+1,:] = genedata
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            break
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
    condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model2.getStateIds()]) 
    condsSDF.to_csv(nmxlsfile1+'S_'+str(nn)+'.txt',sep="\t")
    condsSDF = None
    condsGDF = pd.DataFrame(data=xoutG_all) 
    condsGDF.to_csv(nmxlsfile1+'G_'+str(nn)+'.txt',sep="\t")    
    condsGDF = None    
print(datetime.now())

In [ ]:
numStocCells = 30
ttd_v1 = []
currentDirectory = os.getcwd()
nmxlsfile1 = 'MAi_v1_25cells_72hrs_S_' # No stim
plt.figure(figsize=(7, 4))
for cc in range(numStocCells):
    namer = currentDirectory+'\\Fig5A\\'+nmxlsfile1+str(cc)+'.txt'
    condsDF = pd.read_csv(namer,header=0,index_col=0,sep="\t")
    condsDF = np.double(condsDF.values[:])
    condsDF = condsDF[~np.all(condsDF == 0, axis=1)]
    yy = np.array(condsDF[:,105])
    tt = (np.arange(0,len(condsDF))*30.0/3600.0)
    ttd_v1.append((len(condsDF)-1)*30.0/3600.0)
    plt.plot(tt, yy,linewidth=4)
plt.ylim([0, 400])
plt.xlim([0, 72])
plt.savefig('Fig5A_v1.png')